In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Import Data

In [2]:
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

# Batching 

In [3]:
grouped_data_train = [group for _, group in df_train.groupby('uid')]
grouped_data_test = [group for _, group in df_test.groupby('uid')]

STEP_SIZE = 100
WINDOW_SIZE = 10

class TrajectoryDataset(Dataset):
    def __init__(self, grouped_data):
        self.data = []
        for group in grouped_data:
            if len(group) >= STEP_SIZE:
                # get the first STEP_SIZE location and time data
                xy = group['combined_xy'].values.tolist()[:STEP_SIZE]
                t = group['t'].values.tolist()[:STEP_SIZE]
                # slice the data into several sessions using moving window approach
                self.data.extend([(xy[i:i+WINDOW_SIZE], t[i:i+WINDOW_SIZE])
                                  for i in range(STEP_SIZE - WINDOW_SIZE + 1)])
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        xy_window, t_window = self.data[idx]
        inputs = torch.tensor(xy_window[:-1]) # input sequence of locations
        labels = torch.tensor(xy_window[-1]) # desired predicted location
        positions = torch.tensor(t_window[:-1]) # corresponding input locations' times
        label_positions = torch.tensor(t_window[-1]) # corresponding predicted location's time
        return inputs, labels, positions, label_positions

train_dataset = TrajectoryDataset(grouped_data_train) 
test_dataset = TrajectoryDataset(grouped_data_test)

def collate_fn(batch):
    # Unzip all batch
    inputs_batch, labels_batch, positions_batch, label_positions_batch = zip(*batch)
    
    # Pad the sequence with less length in a batch
    inputs_padded = torch.nn.utils.rnn.pad_sequence(inputs_batch, padding_value=0.0, batch_first=True) 
    labels_padded = torch.tensor(np.array(labels_batch))
    # positions_padded = torch.nn.utils.rnn.pad_sequence(positions_batch, padding_value=0, batch_first=True) 
    # label_positions_padded = torch.tensor(np.array(label_positions_batch))
    
    return inputs_padded, labels_padded # positions_padded, label_positions_padded

BATCH_SIZE = (len(train_dataset) // len(grouped_data_train)) * 10

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

# LSTM
https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/

### Model Building (nn.PyTorch built-in LSTM)

In [16]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, loc_size, embed_dim, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.input_embedding = nn.Embedding(loc_size, embed_dim)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=hidden_size, 
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, loc_size)
        self.num_layers = num_layers
        self.hidden_size = hidden_size

    def forward(self, x):
        # positioanl embedding
        x = self.input_embedding(x)
        
        # initiation
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  
        
        # getting output
        out, _ = self.lstm(x, (h0, c0)) 
        out = self.fc(out[:, -1, :])
        return out

### Training

In [17]:
# Model related param

model = LSTMModel(loc_size=40000, 
                  embed_dim=256, 
                  hidden_size=256, # not sure about this
                  num_layers=2)
optimizer = optim.Adam(model.parameters(), lr=0.0013)
criterion = nn.CrossEntropyLoss()

In [36]:
epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        # inputs = inputs.float().unsqueeze(-1)
        labels = labels.long()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f"Epoch: {epoch}: Average Loss: {total_loss/len(train_dataloader)}")
    
"""
Epoch: 0: Average Loss: 7.053878789760357
Epoch: 1: Average Loss: 4.725554961066872
"""

# Inference

In [19]:
import torch.nn.functional as F

In [32]:
def evaluate_model_with_top_k(model, dataloader, device, k):
    model.eval()
    total_correct = 0
    total_examples = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.long()
            
            logits = model(inputs)
            
            if (BATCH_SIZE > 1):
                for logit_i in range(len(logits)):
                    tolerated_acc = int(labels[logit_i] in torch.topk(logits[logit_i], k)[1])
                    total_correct += tolerated_acc
            else:
                tolerated_acc = int(labels in torch.topk(logits, k)[1])
                total_correct += tolerated_acc

            total_examples += labels.numel()
    
    print(f"Batch Size: {BATCH_SIZE} Top {k} Accuracy: {total_correct/total_examples}, Num Correct: {total_correct}, Num Sample: {total_examples}")
    
    return total_correct/total_examples

In [35]:
lstm_accuracy = evaluate_model_with_top_k(model, test_dataloader, device, 5)

"""
Batch Size: 910 Top 5 Accuracy: 0.30905343392704365, Num Correct: 55629, Num Sample: 179998
"""

# Hyperparameter-tuning

In [ ]:
import optuna


batch_sizes = [int(i) for i in range(5,101,5)]
batch_sizes.append(1)
batch_sizes = sorted(batch_sizes)


def objective(trial):
    # Set up dataloader
    batch_size = trial.suggest_categorical('batch_size', batch_sizes)  ##
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    # Model Parameters
    NUM_CLASS = 40000
    STEP_SIZE = 100
    # EMBED_DIM = trial.suggest_categorical('embed_dim', [64, 128, 256, 512])
    EMBED_DIM = 256
    LAYER_DIM = trial.suggest_int('layer_dim', 1, 6)  ##
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Train the model
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    
    # Model instantiation
    model = LSTMModel(input_dim=1,
                      embed_dim=EMBED_DIM, 
                      layer_dim=LAYER_DIM,
                      output_dim=NUM_CLASS)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    total_loss = 0
    total_samples = 0
    for epoch in range(5):
        model.train()
        for inputs, labels in train_dataloader:
            inputs = inputs.float().unsqueeze(-1)
            labels = labels.long()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * labels.size(0)
            total_samples += labels.size(0)
    
    final_avg_loss = total_loss / total_samples
    return final_avg_loss

# Hyperparameter tuning

# Create a study object and optimize the objective function
study = optuna.create_study()
study.optimize(objective, n_trials=50)

# Result
print('Best parameters:', study.best_params)
print('Best loss:', study.best_value)